In [ ]:
# Eye Blink Detector
import cv2
import numpy as np
import mediapipe as mp
import math

cap = cv2.VideoCapture(0)
mp_face = mp.solutions.face_mesh
mp_draw = mp.solutions.drawing_utils
face_mesh = mp_face.FaceMesh(refine_landmarks=True)

LEFT_EYE = [159,145]
RIGHT_EYE = [386,374]

blink_count = 0
blink_state = False


while True:
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = face_mesh.process(rgb)
    h,w,_ = f.shape

    if res.multi_face_landmarks:
        for face_landmarks in res.multi_face_landmarks:
            mp_draw.draw_landmarks(f,face_landmarks,mp_face.FACEMESH_TESSELATION,mp_draw.DrawingSpec(color=(0,255,0),thickness=1,circle_radius=1),mp_draw.DrawingSpec(color=(0,0,255),thickness=1))

            left_up = face_landmarks.landmark[LEFT_EYE[0]]
            left_down = face_landmarks.landmark[LEFT_EYE[1]]

            right_up = face_landmarks.landmark[RIGHT_EYE[0]]
            right_down = face_landmarks.landmark[RIGHT_EYE[1]]

            l_up = int(left_up.x*w),int(left_up.y*h)
            l_dw = int(left_down.x*w),int(left_down.y*h)
            r_up = int(right_up.x*w),int(right_up.y*h)
            r_dw = int(right_down.x*w),int(right_down.y*h)

            cv2.circle(f,l_up,3,(0,255,0),-1)
            cv2.circle(f,l_dw,3,(0,255,0),-1)
            cv2.circle(f,r_up,3,(0,255,0),-1)
            cv2.circle(f,r_dw,3,(0,255,0),-1)

            left_dist = math.hypot(l_up[0]-l_dw[0],l_up[1]-l_dw[1])
            right_dist = math.hypot(r_up[0]-r_dw[0],r_up[1]-r_dw[1])

            eye_avg = (left_dist+right_dist)/2

            if eye_avg<10 and not blink_state:
                blink_count+=1
                blink_state=True
            elif eye_avg>=10:
                blink_state=False
            
            cv2.putText(f,f"Blink: {blink_count}", (10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
    


    cv2.imshow("Eye Blink Detector",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break


cap.release()
cv2.destroyAllWindows()